In [11]:
df = pd.read_csv("../data/processed/player_weekly_stats.csv")  # IGNORE

In [12]:
df

,season,week,player_id,name,team,team_id,position,opponent,fantasy_points,targets,carries,depth_position,injury_status
0,2023,1,39163,AJ Henning,MIA,19.0,WR,NaN,9.68,1,4,SWR,NaN
1,2023,1,39165,AJ Henning,MIA,19.0,WR,NaN,1.10,0,3,SWR,NaN
2,2023,1,39164,AJ Henning,MIA,19.0,WR,NaN,20.92,0,10,SWR,NaN
3,2023,1,29293,Akeem Spence,NaN,NaN,DT,NaN,-1.20,6,0,NaN,NaN
4,2023,1,29701,Al-Quadin Muhammad,DET,10.0,DE,NaN,2.42,0,3,RDE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,2023,22,30506,Michael Gonzalez,ATL,1.0,OL,NaN,9.30,10,0,OL,NaN
5615,2023,22,37834,Ross Dwelley,SF,27.0,TE,NaN,15.40,0,3,TE,NaN
5616,2023,22,34386,Samaje Perine,CIN,6.0,RB,NaN,5.40,5,0,RB,NaN
5617,2023,22,34286,Sua Opeta,TB,29.0,G,NaN,0.00,1,0,OL,NaN


In [27]:
import os
import subprocess
import pandas as pd
from sqlalchemy import create_engine
import requests
import logging
from dotenv import load_dotenv

load_dotenv()
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

parquet_folder="../data/raw/nflfastr/"
all_dfs = []
seasons = list(range(2021, 2025))

for season in seasons:
    file_path = os.path.join(parquet_folder, f'player_stats_{season}.parquet')
    if not os.path.exists(file_path):
        logging.warning(f'Parquet file for season {season} not found: {file_path}')
        continue
    
    logging.info(f'Loading stats for season {season}')
    df = pd.read_parquet(file_path)

    df = df.rename(columns={
        'season':'season',
        'week':'week',
        'player_id':'player_id',
        'recent_team':'team_abbrevation',
        'targets':'targets',
        'carries':'carries',
        'offense_snaps':'snaps'
    })

    all_dfs.append(df)


In [23]:
df = pd.read_parquet('../data/raw/nflfastr/player_stats_2021.parquet')

In [33]:
def fetch_and_store_weekly_stats(engine, start_year: int, end_year: int, parquet_folder="../data/raw/nflfastr/", output_csv_path="data/processed/player_weekly_stats.csv"):
    all_dfs = []
    seasons = list(range(start_year, end_year + 1))

    for season in seasons:
        file_path = os.path.join(parquet_folder, f"player_stats_{season}.parquet")
        if not os.path.exists(file_path):
            logging.warning(f"⚠️ Parquet file for season {season} not found: {file_path}")
            continue

        logging.info(f"📥 Loading stats for season {season}")
        df = pd.read_parquet(file_path)

        df = df.rename(columns={
            'season': 'season',
            'week': 'week',
            'player_id': 'player_id',
            'recent_team': 'team_abbreviation',
            'targets': 'targets',
            'carries': 'carries',
            'offense_snaps': 'snaps'
        })

        all_dfs.append(df)

    if not all_dfs:
        logging.error("❌ No data loaded from any season.")
        return
    
    return all_dfs

In [35]:
query = """
SELECT
    ws.season,
    ws.week,
    p.player_id,
    p.name,
    t.abbreviation AS team,
    p.team_id,
    p.position,
    CASE 
        WHEN p.team_id = g.home_team_id THEN g.away_team
        ELSE g.home_team
    END AS opponent,
    ws.fantasy_points,
    ws.targets,
    ws.carries,
    d.depth_position,
    i.injury_status
FROM weekly_stats ws
LEFT JOIN players p ON ws.player_id = p.player_id
LEFT JOIN teams t ON p.team_id = t.team_id
LEFT JOIN games g ON ws.season = g.season AND ws.week = g.week
LEFT JOIN depth_chart d ON p.player_id = d.player_id
LEFT JOIN injuries i 
    ON p.player_id = i.player_id 
    AND ws.week = i.week 
    AND ws.season = i.season
ORDER BY ws.season, ws.week, p.name;
"""

# Export
final_df = pd.read_sql(query, engine).drop_duplicates()
# final_df.to_csv(output_csv_path, index=False)
# logging.info(f"Exported to {output_csv_path}")

: 